In [0]:
%pip install xgboost scikit-learn hyperopt joblib pandas

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
# NOTATNIK B: Trening ML (Price-Level Agnostic)

import pandas as pd
import joblib
import numpy as np
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.linear_model import BayesianRidge, LinearRegression 
from sklearn.neighbors import KNeighborsRegressor
from pyspark.sql import functions as F
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
from sklearn.metrics import mean_squared_error

# --- Konfiguracja ---
INPUT_TABLE = "eth_raw_stream_table" 

# Ścieżki zapisu (niezmienione)
PATH_XGB = '/tmp/eth_model_xgboost.joblib'
PATH_BAYES = '/tmp/eth_model_bayes.joblib'
PATH_LINEAR = '/tmp/eth_model_linear.joblib'
PATH_KNN = '/tmp/eth_model_knn.joblib'
N_TRIALS = 20 


def calculate_features_per_block(group):
    """Oblicza wskaźniki dynamiki oparte na logarytmicznych zwrotach."""
    group = group.sort_values("timestamp")
    
    # 1. Baza: Logarytmiczne Zwroty (Usuwamy wpływ bezwzględnej ceny)
    group['log_returns'] = np.log(group['close'] / group['close'].shift(1))
    
    # 2. Wskaźniki Techniczne Oparte na Zwrotach
    group['returns_sma'] = group['log_returns'].rolling(window=12).mean()
    group['returns_std'] = group['log_returns'].rolling(window=12).std()
    
    # 3. Etykieta (Target): Logarytmiczny Zwrot Przyszły (Y)
    future_close = group['close'].shift(-12)
    group['target_log_return'] = np.log(future_close / group['close'])
    
    return group

# ------------------------------------------------------------------

print("1. Ładowanie i Przygotowanie Danych...")

# Wczytanie danych z tabeli
try:
    df_spark = spark.read.table(INPUT_TABLE)
    pandas_df = df_spark.orderBy("timestamp").toPandas()
except Exception as e:
    print(f"❌ Błąd: Nie znaleziono tabeli {INPUT_TABLE}. Uruchom najpierw kod czyszczący!")
    raise e

print(f"Pobrano {len(pandas_df)} surowych wierszy.")

# Aplikujemy funkcję obliczającą cechy (zakładamy, że to jest jeden blok)
pandas_df = calculate_features_per_block(pandas_df)

# Usuwamy wiersze z NaN (powstałe przez rolling i shift)
pandas_df = pandas_df.dropna()

print(f"Po przetworzeniu dostępnych wierszy: {len(pandas_df)}")

if len(pandas_df) < 150:
    print("⚠️ ZA MAŁO DANYCH do sensownego podziału na 3 zbiory! Uruchom Notatnik A.")
else:
    # 2. Podział na Train / Validation / Test (Chronologiczny!)
    
    # Definicja cech (X) i celu (Y)
    # ZMIENIONY WEKTOR CECH: Bez close!
    features_cols = ['returns_sma', 'returns_std', 'volume'] 
    X = pandas_df[features_cols].values
    Y = pandas_df['target_log_return'].values # Cel: Logarytmiczny Zwrot
    
    total_len = len(X)
    train_end = int(total_len * 0.70)
    val_end = int(total_len * 0.85)
    
    # Podział
    X_train, Y_train = X[:train_end], Y[:train_end]
    X_val, Y_val = X[train_end:val_end], Y[train_end:val_end]
    X_test, Y_test = X[val_end:], Y[val_end:]
    
    print(f"Podział zbioru:")
    print(f"  - Treningowy:  {len(X_train)} wierszy")
    print(f"  - Walidacyjny: {len(X_val)} wierszy (do strojenia Hyperopt)")
    print(f"  - Testowy:     {len(X_test)} wierszy (do oceny końcowej)")

    # --- MODEL 1: XGBoost (Optymalizacja na zbiorze WALIDACYJNYM) ---
    print("\n--- Model 1: XGBoost (Champion) ---")
    
    space = {
        'max_depth': hp.choice('max_depth', range(3, 10)),
        'learning_rate': hp.uniform('learning_rate', 0.01, 0.3),
        'n_estimators': hp.choice('n_estimators', range(50, 200, 10)),
        'gamma': hp.uniform('gamma', 0, 0.2),
    }

    def objective(params):
        model = XGBRegressor(**params, random_state=42, n_jobs=-1)
        model.fit(X_train, Y_train) 
        preds = model.predict(X_val)
        rmse = np.sqrt(mean_squared_error(Y_val, preds))
        return {'loss': rmse, 'status': STATUS_OK}

    trials = Trials()
    best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=N_TRIALS, trials=trials)
    
    final_xgb = XGBRegressor(**best, random_state=42, n_jobs=-1)
    final_xgb.fit(X_train, Y_train)
    
    # Ocena na zbiorze TESTOWYM (Ostateczny werdykt)
    test_preds = final_xgb.predict(X_test)
    test_rmse = np.sqrt(mean_squared_error(Y_test, test_preds))
    
    joblib.dump(final_xgb, PATH_XGB)
    print(f"✅ XGBoost zapisany. Test RMSE: {test_rmse:.4f}")
    
    
    # --- MODEL 2: Bayesian Ridge ---
    print("\n--- Model 2: Bayesian Ridge ---")
    final_bayes = BayesianRidge() 
    final_bayes.fit(X_train, Y_train)
    
    bayes_preds = final_bayes.predict(X_test)
    bayes_rmse = np.sqrt(mean_squared_error(Y_test, bayes_preds))
    
    joblib.dump(final_bayes, PATH_BAYES)
    print(f"✅ Bayesian Ridge zapisany. Test RMSE: {bayes_rmse:.4f}")


    # --- MODEL 3: Regresja Liniowa ---
    print("\n--- Model 3: Regresja Liniowa ---")
    final_linear = LinearRegression()
    final_linear.fit(X_train, Y_train)
    
    lin_preds = final_linear.predict(X_test)
    lin_rmse = np.sqrt(mean_squared_error(Y_test, lin_preds))
    
    joblib.dump(final_linear, PATH_LINEAR)
    print(f"✅ Regresja Liniowa zapisana. Test RMSE: {lin_rmse:.4f}")


    # --- MODEL 4: k-NN ---
    print("\n--- Model 4: k-NN ---")
    final_knn = KNeighborsRegressor(n_neighbors=5, weights='distance')
    final_knn.fit(X_train, Y_train)
    
    knn_preds = final_knn.predict(X_test)
    knn_rmse = np.sqrt(mean_squared_error(Y_test, knn_preds))
    
    joblib.dump(final_knn, PATH_KNN)
    print(f"✅ k-NN zapisany. Test RMSE: {knn_rmse:.4f}")

    print("\n==========================================")
    print("PODSUMOWANIE BŁĘDÓW NA ZBIORZE TESTOWYM:")
    print(f"XGBoost: {test_rmse:.4f}")
    print(f"Bayes:   {bayes_rmse:.4f}")
    print(f"Linear:  {lin_rmse:.4f}")
    print(f"k-NN:    {knn_rmse:.4f}")
    print("==========================================")
    print("Modele gotowe do użycia w Notatniku C.")

1. Ładowanie i Przygotowanie Danych...
Pobrano 18877 surowych wierszy.
Po przetworzeniu dostępnych wierszy: 18853
Podział zbioru:
  - Treningowy:  13197 wierszy
  - Walidacyjny: 2828 wierszy (do strojenia Hyperopt)
  - Testowy:     2828 wierszy (do oceny końcowej)

--- Model 1: XGBoost (Champion) ---
100%|██████████| 20/20 [00:01<00:00, 17.54trial/s, best loss: 0.0010024426545248506]
✅ XGBoost zapisany. Test RMSE: 0.0048

--- Model 2: Bayesian Ridge ---
✅ Bayesian Ridge zapisany. Test RMSE: 0.0048

--- Model 3: Regresja Liniowa ---
✅ Regresja Liniowa zapisana. Test RMSE: 0.0048

--- Model 4: k-NN ---
✅ k-NN zapisany. Test RMSE: 0.0050

PODSUMOWANIE BŁĘDÓW NA ZBIORZE TESTOWYM:
XGBoost: 0.0048
Bayes:   0.0048
Linear:  0.0048
k-NN:    0.0050
Modele gotowe do użycia w Notatniku C.


In [0]:
# NOTATNIK C: ULTIMATE COMPARATOR (5 Modeli)

import time
import pandas as pd
import joblib
import numpy as np
from datetime import datetime
from pyspark.sql import functions as F
from sklearn.metrics import mean_squared_error, mean_absolute_error

# --- Konfiguracja Ścieżek ---
# WAŻNE: Używamy tabeli oczyszczonej, która jest bardziej stabilna
INPUT_TABLE = "eth_raw_stream_table" 
REFRESH_RATE = 5 
# Ścieżki do modeli utworzonych przez Notatnik B
PATH_XGB = '/tmp/eth_model_xgboost.joblib'
PATH_BAYES = '/tmp/eth_model_bayes.joblib'
PATH_LINEAR = '/tmp/eth_model_linear.joblib'
PATH_KNN = '/tmp/eth_model_knn.joblib'
HISTORY_WINDOW = 60 # Okno historii do obliczeń statystycznych

# --- STATE: Klasa do Śledzenia Błędów (Bez zmian) ---
class ErrorTracker:
    def __init__(self):
        self.actuals = []
        self.preds = []
    def update(self, actual, predicted):
        self.actuals.append(actual)
        self.preds.append(predicted)
        # Trzymamy tylko ostatnie 100 próbek do statystyk
        if len(self.actuals) > 100:
            self.actuals.pop(0)
            self.preds.pop(0)
    def get_rmse(self):
        if len(self.actuals) < 2: return 999.9
        return np.sqrt(mean_squared_error(self.actuals, self.preds))
    def get_mae(self):
        if len(self.actuals) < 2: return 999.9
        return mean_absolute_error(self.actuals, self.preds)

# --- FUNKCJE POMOCNICZE (AIC/BIC) (Bez zmian) ---
def calculate_aic_bic(residuals, n, k):
    """Oblicza AIC i BIC na podstawie reszt, zakładając rozkład normalny błędów."""
    rss = np.sum(residuals**2)
    if rss <= 0 or n <= 0: return 0, 0
    # Log-Likelihood dla regresji:
    log_likelihood = -n/2 * (np.log(2 * np.pi) + np.log(rss/n) + 1)
    aic = 2*k - 2*log_likelihood
    bic = k*np.log(n) - 2*log_likelihood
    return aic, bic

# --- 1. ŁADOWANIE MODELI I INICJALIZACJA ---
trackers = {name: ErrorTracker() for name in ["XGBoost", "Bayes", "Linear", "kNN", "Stoch"]}
models = {}
path_map = {
    "XGBoost": PATH_XGB, 
    "Bayes": PATH_BAYES, 
    "Linear": PATH_LINEAR, 
    "kNN": PATH_KNN
}

print("1. Ładowanie 4 Modeli z Dysku...")
for name, path in path_map.items():
    try:
        models[name] = joblib.load(path)
        print(f"✅ {name} załadowany.")
    except:
        print(f"❌ Brak modelu: {name} (Uruchom Notatnik B!)")

# 2. Główna Funkcja Analizy (Micro-Batch)
def run_comparison_analysis(last_predictions_dict):
    try:
        df = spark.read.table(INPUT_TABLE)
        if df.isEmpty() or df.count() < 20: return None
        
        # Pobieramy dane do Pandas (limitujemy dla szybkości)
        pdf = df.orderBy(F.col("timestamp").desc()).limit(100).toPandas()
        pdf = pdf.sort_values("timestamp")
        
        # --- NOWA INŻYNIERIA CECH (Dopasowanie do Notatnika B!) ---
        pdf['log_returns'] = np.log(pdf['close'] / pdf['close'].shift(1))
        pdf['returns_sma'] = pdf['log_returns'].rolling(window=12).mean()
        pdf['returns_std'] = pdf['log_returns'].rolling(window=12).std()
        # Wymagany cel do AIC/BIC (nie do predykcji)
        pdf['target_log_return'] = np.log(pdf['close'].shift(-12) / pdf['close']) 
        
        pdf = pdf.dropna() # Usuwamy NaN powstałe w inżynierii cech
        
        if pdf.empty: return None

        current_price = pdf['close'].iloc[-1]
        current_time = pdf['timestamp'].iloc[-1]
        # Zmienność (Sigma) dla modelu stochastycznego (na podstawie zwrotów)
        sigma = pdf['log_returns'].std() if not np.isnan(pdf['log_returns'].std()) else 0.001
        
        # Przygotowanie wejścia dla modeli ML (X)
        # KOLEJNOŚĆ I NAZWY MUSZĄ BYĆ ZGODNE Z NOTATNIKIEM B!
        X_input = pdf[['returns_sma', 'returns_std', 'volume']].fillna(0).iloc[[-1]]
        X_input_array = X_input.values 
        
        preds = {}
        
        # --- PREDYKCJE (MODELE 1-4) ---
        # Modele przewidują LOGARYTMICZNY ZWROT, konwersja: P(t+1) = P(t) * exp(log_return)
        for name in ['XGBoost', 'Bayes', 'Linear', 'kNN']:
            if name in models:
                log_return_pred = models[name].predict(X_input_array)[0]
                preds[name] = current_price * np.exp(log_return_pred)
            else:
                preds[name] = current_price # Fallback

        # --- 5. MODEL STOCHASTYCZNY (MONTE CARLO) ---
        # Używamy predykcji log-zwrotu ML jako DRYFTU
        model_for_drift = models.get('Bayes', models.get('XGBoost'))
        if model_for_drift is not None:
             drift_ml_log_return = model_for_drift.predict(X_input_array)[0]
        else:
             drift_ml_log_return = 0.0 # Zerowy drift jeśli brak modeli ML
        
        n_sims = 1000
        jump_prob = 0.05 
        
        # Symulacja bazuje na log-zwrotach
        brownian_log = np.random.normal(0, sigma, n_sims)
        jumps = np.random.choice([0, 1], size=n_sims, p=[1-jump_prob, jump_prob])
        jump_mag_log = np.random.normal(0, 3 * sigma, n_sims) * jumps
        
        # Całkowity Log-Zwrot dla symulacji:
        total_log_return_sim = drift_ml_log_return + brownian_log + jump_mag_log
        
        # Konwersja do cen
        simulated_prices = current_price * np.exp(total_log_return_sim)
        preds['Stoch'] = np.mean(simulated_prices)

        # --- AKTUALIZACJA TRACKERÓW (Bez zmian, bo używamy cen) ---
        if last_predictions_dict:
            for name, old_pred in last_predictions_dict.items():
                trackers[name].update(current_price, old_pred)

        # --- OBLICZANIE AIC/BIC (Na podstawie Log-Zwrotów!) ---
        aic_bayes, bic_bayes = 0, 0
        aic_linear, bic_linear = 0, 0
        
        # Oczekiwany wynik to teraz 'target_log_return'
        hist_y = pdf['target_log_return'].tail(HISTORY_WINDOW).values 
        # Cechy wejściowe
        hist_X = pdf[['returns_sma', 'returns_std', 'volume']].fillna(0).tail(HISTORY_WINDOW)
        k_params = 3 # Liczba cech: returns_sma, returns_std, volume

        if len(hist_y) > k_params:
            # Dla Bayesa
            if 'Bayes' in models:
                try:
                    hist_log_return_preds = models['Bayes'].predict(hist_X)
                    # Rezydualna to (Rzeczywisty Log-Zwrot - Prognozowany Log-Zwrot)
                    residuals = hist_y - hist_log_return_preds 
                    aic_bayes, bic_bayes = calculate_aic_bic(residuals, len(hist_y), k_params)
                except: pass

            # Dla Liniowego
            if 'Linear' in models:
                try:
                    hist_log_return_preds = models['Linear'].predict(hist_X)
                    # Rezydualna to (Rzeczywisty Log-Zwrot - Prognozowany Log-Zwrot)
                    residuals = hist_y - hist_log_return_preds
                    aic_linear, bic_linear = calculate_aic_bic(residuals, len(hist_y), k_params)
                except: pass
            
        return {
            "ts": current_time,
            "price": current_price,
            "preds": preds,
            "stats": {
                "Bayes": (aic_bayes, bic_bayes),
                "Linear": (aic_linear, bic_linear)
            }
        }

    except Exception as e:
        # print(f"Błąd w analizie: {e}") # Odkomentuj do debugowania
        return None

# --- Pętla Główna (Bez zmian) ---
print(f"🚀 Start Systemu 5 Modeli (Champion/Challenger).")
last_preds_dict = None

try:
    while True:
        res = run_comparison_analysis(last_preds_dict)
        
        if res:
            # Pobieramy RMSE dla wszystkich modeli
            rmses = {name: t.get_rmse() for name, t in trackers.items()}
            # Wybieramy najlepszy model (najmniejsze RMSE)
            winner = min(rmses, key=rmses.get)
            
            ts_str = res['ts'].strftime('%H:%M:%S')
            print(f"\n[{ts_str}] CENA: {res['price']:.2f} USD")
            print("-" * 105)
            print(f"{'MODEL':<18} | {'PROGNOZA':<10} | {'RMSE (Błąd)':<15} | {'MAE':<8} | {'AIC':<8} | {'BIC':<8}")
            print("-" * 105)
            
            for name in ["XGBoost", "Bayes", "Linear", "kNN", "Stoch"]:
                marker = "👑" if name == winner else "  "
                rmse = rmses.get(name, 0.0)
                mae = trackers[name].get_mae()
                
                # Pobieramy AIC/BIC jeśli dostępne
                aic_val, bic_val = res['stats'].get(name, (0, 0))
                
                # Formatowanie tabeli
                aic_str = f"{aic_val:.1f}" if aic_val != 0 else "-"
                bic_str = f"{bic_val:.1f}" if bic_val != 0 else "-"
                
                print(f"{marker} {name:<15} | {res['preds'].get(name, 0.0):<10.2f} | {rmse:<15.4f} | {mae:<8.4f} | {aic_str:<8} | {bic_str:<8}")
                
            print("-" * 105)
            print(f"🏆 CHAMPION: {winner}")
            
            # Zapisujemy obecne prognozy do weryfikacji w następnej pętli
            last_preds_dict = res['preds']
            
        else:
            print(".", end="", flush=True)

        time.sleep(REFRESH_RATE)

except KeyboardInterrupt:
    print("Zatrzymano.")

1. Ładowanie 4 Modeli z Dysku...
✅ XGBoost załadowany.
✅ Bayes załadowany.
✅ Linear załadowany.
✅ kNN załadowany.
🚀 Start Systemu 5 Modeli (Champion/Challenger).


/databricks/python/lib/python3.12/site-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but BayesianRidge was fitted without feature names
  warnings.warn(
/databricks/python/lib/python3.12/site-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(



[21:23:42] CENA: 3056.20 USD
---------------------------------------------------------------------------------------------------------
MODEL              | PROGNOZA   | RMSE (Błąd)     | MAE      | AIC      | BIC     
---------------------------------------------------------------------------------------------------------
👑 XGBoost         | 3055.97    | 999.9000        | 999.9000 | -        | -       
   Bayes           | 3055.86    | 999.9000        | 999.9000 | -627.7   | -621.4  
   Linear          | 3055.83    | 999.9000        | 999.9000 | -627.2   | -620.9  
   kNN             | 3055.01    | 999.9000        | 999.9000 | -        | -       
   Stoch           | 3055.82    | 999.9000        | 999.9000 | -        | -       
---------------------------------------------------------------------------------------------------------
🏆 CHAMPION: XGBoost


/databricks/python/lib/python3.12/site-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but BayesianRidge was fitted without feature names
  warnings.warn(
/databricks/python/lib/python3.12/site-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(



[21:23:48] CENA: 3055.92 USD
---------------------------------------------------------------------------------------------------------
MODEL              | PROGNOZA   | RMSE (Błąd)     | MAE      | AIC      | BIC     
---------------------------------------------------------------------------------------------------------
👑 XGBoost         | 3055.69    | 999.9000        | 999.9000 | -        | -       
   Bayes           | 3055.58    | 999.9000        | 999.9000 | -627.6   | -621.3  
   Linear          | 3055.56    | 999.9000        | 999.9000 | -627.1   | -620.9  
   kNN             | 3058.01    | 999.9000        | 999.9000 | -        | -       
   Stoch           | 3055.62    | 999.9000        | 999.9000 | -        | -       
---------------------------------------------------------------------------------------------------------
🏆 CHAMPION: XGBoost


...

/databricks/python/lib/python3.12/site-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but BayesianRidge was fitted without feature names
  warnings.warn(
/databricks/python/lib/python3.12/site-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(



[21:39:14] CENA: 3053.86 USD
---------------------------------------------------------------------------------------------------------
MODEL              | PROGNOZA   | RMSE (Błąd)     | MAE      | AIC      | BIC     
---------------------------------------------------------------------------------------------------------
👑 XGBoost         | 3053.63    | 0.5067          | 0.4053   | -        | -       
   Bayes           | 3053.52    | 0.5618          | 0.4726   | -762.8   | -756.5  
   Linear          | 3053.49    | 0.5750          | 0.4871   | -761.2   | -754.9  
   kNN             | 3052.89    | 3.1352          | 1.3786   | -        | -       
   Stoch           | 3053.52    | 0.5641          | 0.4743   | -        | -       
---------------------------------------------------------------------------------------------------------
🏆 CHAMPION: XGBoost


/databricks/python/lib/python3.12/site-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but BayesianRidge was fitted without feature names
  warnings.warn(
/databricks/python/lib/python3.12/site-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(



[21:39:20] CENA: 3053.87 USD
---------------------------------------------------------------------------------------------------------
MODEL              | PROGNOZA   | RMSE (Błąd)     | MAE      | AIC      | BIC     
---------------------------------------------------------------------------------------------------------
👑 XGBoost         | 3053.64    | 0.5072          | 0.4075   | -        | -       
   Bayes           | 3053.53    | 0.5627          | 0.4747   | -761.8   | -755.5  
   Linear          | 3053.50    | 0.5760          | 0.4891   | -760.1   | -753.8  
   kNN             | 3053.29    | 3.1363          | 1.3833   | -        | -       
   Stoch           | 3053.57    | 0.5650          | 0.4765   | -        | -       
---------------------------------------------------------------------------------------------------------
🏆 CHAMPION: XGBoost


com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:139)
	at scala.Option.getOrElse(Option.scala:201)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:139)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:136)
	at scala.collection.immutable.Range.foreach(Range.scala:192)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:136)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:721)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:441)
	at scala.Option.getOrElse(Option.scala:201)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:441)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.can